# Data Preparation

My initial task is to prepare and get myself familiar with the dataset [Suomi24-toxicity-annotated](https://huggingface.co/datasets/TurkuNLP/Suomi24-toxicity-annotated).

In [ ]:
from datasets import load_dataset

dataset = load_dataset(
    "TurkuNLP/Suomi24-toxicity-annotated",
    split="test",
)

Now that dataset is downloaded, it's a good idea to see in what proportion data is.

In [ ]:
import seaborn as sns

sns.countplot(y=dataset["label"], palette="flare")

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = 'TurkuNLP/gpt3-finnish-small'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)